# REPORT: Predicting Risk of Heart Disease from Accessible Health Metrics

## Introduction:

According to the Public Health Agency of Canada, heart disease is the second leading cause of death in Canada, with approximately 1 in 12 Canadian adults over 20 living with a diagnosis. These metrics highlight the importance of knowing the risk factors and having access to medical advice. However, a shortage of physicians in Canada is causing a lack of available health care (Flood et al., 2023). Non-healthcare professionals do not have the means to properly self-evaluate symptoms, therefore our project seeks to help the general population to make informative decisions about heart disease symptoms that are self-monitored or easily accessible.


Thus we ask, is it possible to classify individuals into levels of heart disease risk (low risk, moderate risk, or high risk) based on blood pressure, cholesterol, heart rate and chest pain?


Our analysis will use the Heart Disease dataset from the Cleveland database for heart disease (Andras et al., 1988). This database consists of 303 patients without history of heart disease, who were admitted to the Cleveland Clinic between 1981 and 1984. 



In [ ]:
# Please uncomment the following cell to install the altair in case your package is not up-to-date

In [ ]:
# pip install -U altair

In [ ]:
import altair as alt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.utils import resample
from sklearn.pipeline import make_pipeline

## Loading the Heart Disease Dataset

Here, the Heart Disease dataset from UC Irvine Machine Learning Repository is loaded into the notebook. We also rename the variables within the 'diagnosis' and 'chest pain' columns into more meaningful names.  
> For the 'diagnosis' column indicating the risk level of a heart disease diagnosis : 
<br>
> 0,1 is represented as "low-risk diagnosis"
<br>
> 2,3 is represented as "moderate-risk diagnosis"
<br>
> 4 is represented as "high-risk diagnosis"

In [ ]:
#load the data
heartdisease = pd.read_csv("https://archive.ics.uci.edu/static/public/45/data.csv")

#extract and rename the columns with data that is required
heartdisease = heartdisease[["chol","cp","thalach","trestbps","num"]]
heartdisease.rename(columns = {
                          "chol" : "cholesterol", 
                          "cp":"type_chestpain",
                          "thalach" : "max_heart_rate",
                          "num" : "diagnosis",
                          "trestbps" : "resting_bp"
}, inplace = True)

# renaming variables in the 'diagnosis' column with readable and explicit labels
heartdisease['diagnosis'] = heartdisease['diagnosis'].replace([0,1], "low-risk heart disease")
heartdisease['diagnosis'] = heartdisease['diagnosis'].replace([2,3], "moderate-risk heart disease")
heartdisease['diagnosis'] = heartdisease['diagnosis'].replace([4], "high-risk heart disease")

# renaming variables in the 'chest pain' column with readable and explicit labels
heartdisease['type_chestpain'] = heartdisease['type_chestpain'].replace(
    [1,2,3,4],
    ["type1","type2","type3","type4"])

heartdisease

## Wrangling the Data

For our preliminary analysis, we have shown that the unique labels in the 'diagnosis' column are unbalanced, so we have resolved this by oversampling the rare classes, which are the "moderate-" and "high-risk disease" classes.

In [ ]:
# unbalanced diagnoses, must resample
heartdisease['diagnosis'].value_counts(normalize = True)

In [ ]:
np.random.seed(5432)

# separate the classes out into their own data frames by filtering
high_risk = heartdisease[heartdisease["diagnosis"] == "high-risk heart disease"]
moderate_risk = heartdisease[heartdisease["diagnosis"] == "moderate-risk heart disease"]
low_risk = heartdisease[heartdisease["diagnosis"] == "low-risk heart disease"]

# increase the number of rare observations to the same as the number of non-rare observations
high_risk_upsample = resample(high_risk, n_samples = low_risk.shape[0])
moderate_risk_upsample = resample(moderate_risk, n_samples = low_risk.shape[0])

# combine observations show that the classes are now balanced
heartdisease = pd.concat((high_risk_upsample, moderate_risk_upsample, low_risk))
heartdisease['diagnosis'].value_counts(normalize = True)

## Split into Training and Testing Data sets

We have split the data set so that 75% of our original data set ends up in the training set. We will also set the stratify argument to the categorical label variable, 'diagnosis', to ensure that the training and testing subsets contain the right proportions of each category of observation.

In [ ]:
# split into train and test
heartdisease_train, heartdisease_test = train_test_split(
    heartdisease, train_size=0.75, stratify = heartdisease['diagnosis']
)

## Summary of the Categorical and Continuous Variables

Here we show an overall summary of the variables: 
<br>
**For the categorical variables**: the number of observations (count), the unique labels, the most frequent value (top) and the frequency of the top value.
<br>
**For the continuous variables**: the number of observations (count), the mean and standard deviation of the observations, the min and the max values and the 25th, 50th, and 75th percentiles of each column of variables for the training set. 

In [ ]:
# Summary of the categorical variables
heart_disease_categorical = heart_disease_train.drop(columns = ["cholesterol","max_heart_rate","resting_bp"])
heart_disease_categorical.describe()

In [ ]:
# Summary of the continuous variables
heart_disease_continuous = heart_disease_train.drop(columns = ["type_chestpain","diagnosis"])
heart_disease_continuous.describe()

## Distribution of the Variables in Training Set

Here we visualize the distribution of the measurements for each variable on the training set along with the diagnosis associated with the measurements with bar graphs. The diagnosis is represented with distinct colors. 

In [ ]:
# distribution of blood pressure measurements
bp_hist = alt.Chart(heart_disease_train).mark_bar().encode(
    x=alt.X("resting_bp:Q", bin = True).title("Blood Pressure"),
    y=alt.Y("count()").stack(False),
    color="diagnosis:N"
).properties(
    title = "Distribution of Blood Pressure"
)

# distribution of cholesterol measurements
chol_hist = alt.Chart(heart_disease_train).mark_bar().encode(
    x=alt.X("cholesterol:Q", bin = True).title("Cholesterol"),
    y=alt.Y("count()").stack(False),
    color = "diagnosis:N"
).properties(
    title = "Distribution of Cholesterol"
)

# distribution of chest pain type measurements
cp_hist = alt.Chart(heart_disease_train).mark_bar().encode(
    x=alt.X("type_chestpain").title("Chest Pain Type"),
    y=alt.Y("count()").stack(False),
    color = "diagnosis:N"
).properties(
    width=300,
    height=300,
    title = "Distribution of Chest Pain Type"
)

# distribution of heart rate measurements
hr_hist = alt.Chart(heart_disease_train).mark_bar().encode(
    x=alt.X("max_heart_rate:Q", bin = True).title("Heart Rate"),
    y=alt.Y("count()").stack(False),
    color = "diagnosis:N"
).properties(
    title = "Distribution of Heart Rate"
)


combined_charts = bp_hist | chol_hist | cp_hist | hr_hist
combined_charts

## Standardize the Data Set

We created a pipeline that specifies the preprocessing steps and the K-neighbors classifier. Then we defined the parameter grid to show the range of K-values that will be tuned. GridSearchCV is utilized to estimate the classifier accuracy for the range. Following, we executed the grid search by passing the training data to the fit method, and visualized the results with a line plot. 

In [ ]:
# create a preprocessor
heartdisease_preprocessor = make_column_transformer(
    (StandardScaler(), ["cholesterol", "max_heart_rate", "resting_bp"]),
    remainder = 'passthrough',
    verbose_feature_names_out = False
)

In [ ]:
# create a pipeline
heartdisease_pipe = make_pipeline(heartdisease_preprocessor, KNeighborsClassifier())

# create parameter grid
param_grid = {
    "kneighborsclassifier__n_neighbors": range(2, 20),
}

# estimate the classifier accuracy for the range of values
gridsearch = GridSearchCV(
    estimator = heartdisease_pipe,
    param_grid = param_grid,
    cv = 5,
    n_jobs = -1,
    return_train_score = True
)

cv_results = pd.DataFrame(gridsearch.fit(heartdisease_train[['cholesterol','max_heart_rate','resting_bp']], 
                                         heartdisease_train['diagnosis'])
                          .cv_results_)

# plot the accuracy of the K-values with line-plot
crossvalplot = alt.Chart(cv_results).mark_line(point = True).encode(
    x = alt.X("param_kneighborsclassifier__n_neighbors").title("K Value"),
    y = alt.Y("mean_test_score").title("Accuracy").scale(zero = False)
)

crossvalplot

We show that the best K-parameter is 2, because it provides optimal accuracy, it is not too large to become prohibitive of the model and changing the value to a nearby one would not change the accuracy too much. 

In [ ]:
# create a new model object for the best parameter value
knn_spec = KNeighborsClassifier(n_neighbors = 2)
heartdisease_fit = make_pipeline(heartdisease_preprocessor, knn_spec).fit(heartdisease_train[['cholesterol','max_heart_rate','resting_bp']], heartdisease_train['diagnosis'])

heartdisease_fit

In [ ]:
# add predictions to dataframe
heartdisease_predictions = heartdisease_test.assign(predicted = heartdisease_fit.predict(heartdisease_test[['cholesterol', 'max_heart_rate', 'resting_bp']]))
heartdisease_predictions


## Accuracy

In [ ]:
# test model's accuracy 
heartdisease_acc = heartdisease_fit.score(
    X = heartdisease_test[['cholesterol','max_heart_rate','resting_bp']],
    y = heartdisease_test['diagnosis']
)

heartdisease_acc

In [ ]:
# test models accuracy
correct_preds = heartdisease_predictions[
    heartdisease_predictions['diagnosis'] == heartdisease_predictions['predicted']
]

correct_preds.shape[0] / heartdisease_predictions.shape[0]

In [ ]:
pd.crosstab(
    heartdisease_predictions['diagnosis'],
    heartdisease_predictions['predicted']
)